## Investigating the Zillow Housing Price Dataset

N. Ranjan & R. Mattson | CSCI 6430* | Mar 11, 2021




TODO: 
- check file names

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score as r2score
from sklearn.preprocessing import StandardScaler
#from sklearn.metrics import mean_absolute_percentage_error as mape
#from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import Lasso,Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_validate
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import OneHotEncoder

###  Retrieve Data and Prepare Dataset

In [2]:
#TODO - get from google docs??
DF1B = pd.read_csv(r'Datasets/City_MedianRentalPrice_1Bedroom.csv')
DF2B = pd.read_csv(r'Datasets/City_MedianRentalPrice_2Bedroom.csv')
DF3B = pd.read_csv(r'Datasets/City_MedianRentalPrice_3Bedroom.csv')
DF4B = pd.read_csv(r'Datasets/City_MedianRentalPrice_4Bedroom.csv')

In [3]:
# remove columns with useless information - These were only indexes
DF1B.drop(columns=['SizeRank','Unnamed: 0'],inplace=True)
DF2B.drop(columns=['SizeRank','Unnamed: 0'],inplace=True)
DF3B.drop(columns=['SizeRank','Unnamed: 0'],inplace=True)
DF4B.drop(columns=['SizeRank','Unnamed: 0'],inplace=True)

In [4]:
# record the number of bedrooms within the frame
DF1B.insert(0, 'BedroomsSq', 1)
DF2B.insert(0, 'BedroomsSq', 2)
DF3B.insert(0, 'BedroomsSq', 3)
DF4B.insert(0, 'BedroomsSq', 4)
#BE CAREFUL WITH INSERTING BEDROOMS
insertBedrooms = np.where(DF1B.columns=='2010-02')[0][0] 
print(int(insertBedrooms))
DF1B.insert(int(insertBedrooms), 'Bedrooms', 1)
DF2B.insert(int(insertBedrooms), 'Bedrooms', 2)
DF3B.insert(int(insertBedrooms), 'Bedrooms', 3)
DF4B.insert(int(insertBedrooms), 'Bedrooms', 4)

5


In [5]:
frames = [DF1B,DF2B,DF3B,DF4B]
result = pd.concat(frames)
result = result.sample(frac=1) #Shuffle!
result = result.reset_index(drop=True) #Reset Index

In [6]:
result

,BedroomsSq,RegionName,State,Metro,CountyName,Bedrooms,2010-02,2010-03,2010-04,2010-05,...,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12
0,2,Springfield,OH,Springfield,Clark County,2,NaN,NaN,NaN,NaN,...,665.0,685.0,714.0,714.0,685.0,695.0,697.5,675.0,650.0,675.0
1,4,Simpsonville,SC,Greenville-Anderson-Mauldin,Greenville County,4,NaN,NaN,NaN,NaN,...,1812.5,1770.0,1795.0,1795.0,1795.0,1695.0,1650.0,1695.0,1800.0,1850.0
2,2,Cranston,RI,Providence-Warwick,Providence County,2,NaN,NaN,NaN,NaN,...,1397.5,1397.5,1300.0,1200.0,1300.0,1350.0,1300.0,1300.0,1200.0,1250.0
3,1,Kansas City,KS,Kansas City,Wyandotte County,1,NaN,NaN,NaN,NaN,...,830.5,863.0,747.0,744.0,785.0,735.0,705.0,785.0,747.5,695.0
4,4,Riverview,FL,Tampa-St. Petersburg-Clearwater,Hillsborough County,4,NaN,NaN,NaN,NaN,...,1712.5,1750.0,1750.0,1750.0,1750.0,1750.0,1760.0,1750.0,1752.5,1745.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3384,2,Shrewsbury,MA,Worcester,Worcester County,2,NaN,NaN,NaN,NaN,...,1790.5,1890.0,1699.0,1727.5,1699.0,1725.0,1857.0,1867.0,1701.5,1762.0
3385,1,Surfside,FL,Miami-Fort Lauderdale-West Palm Beach,Miami-Dade County,1,NaN,NaN,NaN,NaN,...,1875.0,1850.0,1750.0,1800.0,1800.0,1750.0,1750.0,1800.0,1800.0,1850.0
3386,1,Brookfield,WI,Milwaukee-Waukesha-West Allis,Waukesha County,1,NaN,NaN,NaN,NaN,...,1385.0,1385.0,1386.0,1386.0,1395.0,1405.0,1365.0,1395.0,1395.0,1395.0
3387,1,Dedham,MA,Boston-Cambridge-Newton,Norfolk County,1,NaN,NaN,NaN,NaN,...,2392.5,2345.0,2347.5,2347.5,2343.5,2433.0,2334.5,1985.0,1950.0,1725.0


In [7]:
# result.to_csv(r'Datasets\City_MedianRentalPrice_AllHomes.csv')
result.to_csv(r'Datasets/City_MedianRentalPrice_AllHomes_ALTERED.csv')


In [8]:
del DF1B,DF2B,DF3B,DF4B
del result,frames
#Just saving memory, let's continue with our final dataset

### Preproccess Data

In [9]:
# DF = pd.read_csv(r'Datasets\City_MedianRentalPrice_AllHomes.csv')
DF = pd.read_csv(r'Datasets/City_MedianRentalPrice_AllHomes_ALTERED.csv',index_col=0)
# TrainDF = DF.iloc[:,:-1].copy() # Rest of the columns come here as training data
# TestDF = DF.iloc[:,-1].copy()   # We predicting the last column
# del DF


In [10]:
ST = DF['State'].value_counts()
ST

CA    564
FL    426
TX    248
GA    167
VA    146
MA    135
WA    124
IL    103
MD    102
NJ     98
CO     82
NC     79
CT     79
OH     76
MI     75
NY     74
PA     70
MN     52
SC     52
MO     49
IN     46
OR     43
UT     37
AZ     35
KS     35
IA     32
OK     29
LA     27
TN     26
WI     23
RI     21
HI     20
NV     20
AL     18
ID     17
KY     17
NM     16
MS     15
AR     14
ND     14
MT     13
DE     13
NE     12
WV     10
AK      8
NH      6
SD      6
ME      4
DC      4
WY      4
VT      3
Name: State, dtype: int64

In [11]:
lb=LabelEncoder()
lb.fit(DF['State'])

LabelEncoder()

In [12]:

RecordAnalysisDF = pd.DataFrame(columns=['RunNumber','Title','Algorithm','R2Mean','R2Median','RMSEMean','RMSEMedian']) #Will save results for later
runNo = 1

In [13]:
isBedStateOptimized = True #LEAVE THIS TRUE

In [14]:
cut_off = 0.25 # if number of nulls > 25%, just remove the column.
#We got lucky here as the null values always increased the further we go in the past 
row_count = DF.index.size
NullCountDF = pd.DataFrame({'Null_Count':DF.isnull().sum().to_numpy(),'Col_Name':DF.columns.to_numpy()})
colsToKeep = NullCountDF[NullCountDF.Null_Count<=row_count*cut_off].Col_Name.to_numpy() #columns to keep,rest have too many nulls
DF = DF[colsToKeep].copy() 
print(DF.isnull().sum().to_numpy())
print(colsToKeep)

[  0   0   0   0   0   0 829 800 771 741 695 621 571 510 465 412 361 306
 249 209 144 107  60   0   0   0   0   0   0   0   0   0   0   0   0]
['BedroomsSq' 'RegionName' 'State' 'Metro' 'CountyName' 'Bedrooms'
 '2017-08' '2017-09' '2017-10' '2017-11' '2017-12' '2018-01' '2018-02'
 '2018-03' '2018-04' '2018-05' '2018-06' '2018-07' '2018-08' '2018-09'
 '2018-10' '2018-11' '2018-12' '2019-01' '2019-02' '2019-03' '2019-04'
 '2019-05' '2019-06' '2019-07' '2019-08' '2019-09' '2019-10' '2019-11'
 '2019-12']


In [15]:
#Removing Nulls
def RemoveNulls(TheDF):
   print('Removing Nulls...')
   for col in TheDF.columns:
      for x in TheDF[TheDF[col].isnull()].index: #Going through all columns
         beds = TheDF.loc[x]['Bedrooms']
         state = TheDF.loc[x]['State']
         #print)
         if(isBedStateOptimized == True):
            valFill = TheDF[(TheDF['Bedrooms'] == beds) & (TheDF['State']==state)][col] #Bedroom and state
            if np.all(np.isnan(valFill)):
               valFill = TheDF[(TheDF['Bedrooms'] == beds)][col] #Check if we can do only with bedroom as this is more correlated than state
            if np.all(np.isnan(valFill)):
               valFill = TheDF[(TheDF['State'] == beds)][col] #If only bedrooms didnt work, try with State
            if np.all(np.isnan(valFill)):
               valFill =TheDF[col] #If we get null for those too..just take median of whole column
            valFill = np.nanmedian(valFill)
            TheDF.loc[x,col] = valFill
         else:
            valFill =TheDF[col]
            valFill = np.nanmedian(valFill)
            TheDF.loc[x,col] = valFill
   print('Nulls Removed.')

In [16]:
RemoveNulls(DF)

Removing Nulls...
Nulls Removed.


In [17]:
DF

,BedroomsSq,RegionName,State,Metro,CountyName,Bedrooms,2017-08,2017-09,2017-10,2017-11,...,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12
0,2,Springfield,OH,Springfield,Clark County,2,827.50,860.00,812.5,850.0,...,665.0,685.0,714.0,714.0,685.0,695.0,697.5,675.0,650.0,675.0
1,4,Simpsonville,SC,Greenville-Anderson-Mauldin,Greenville County,4,1750.00,1725.00,1695.0,1650.0,...,1812.5,1770.0,1795.0,1795.0,1795.0,1695.0,1650.0,1695.0,1800.0,1850.0
2,2,Cranston,RI,Providence-Warwick,Providence County,2,1300.00,1300.00,1250.0,1100.0,...,1397.5,1397.5,1300.0,1200.0,1300.0,1350.0,1300.0,1300.0,1200.0,1250.0
3,1,Kansas City,KS,Kansas City,Wyandotte County,1,918.00,895.00,917.5,890.0,...,830.5,863.0,747.0,744.0,785.0,735.0,705.0,785.0,747.5,695.0
4,4,Riverview,FL,Tampa-St. Petersburg-Clearwater,Hillsborough County,4,1650.00,1675.00,1612.5,1599.0,...,1712.5,1750.0,1750.0,1750.0,1750.0,1750.0,1760.0,1750.0,1752.5,1745.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3384,2,Shrewsbury,MA,Worcester,Worcester County,2,2000.00,1975.00,1971.0,1987.5,...,1790.5,1890.0,1699.0,1727.5,1699.0,1725.0,1857.0,1867.0,1701.5,1762.0
3385,1,Surfside,FL,Miami-Fort Lauderdale-West Palm Beach,Miami-Dade County,1,1825.00,1800.00,1800.0,1800.0,...,1875.0,1850.0,1750.0,1800.0,1800.0,1750.0,1750.0,1800.0,1800.0,1850.0
3386,1,Brookfield,WI,Milwaukee-Waukesha-West Allis,Waukesha County,1,921.25,875.00,860.0,887.5,...,1385.0,1385.0,1386.0,1386.0,1395.0,1405.0,1365.0,1395.0,1395.0,1395.0
3387,1,Dedham,MA,Boston-Cambridge-Newton,Norfolk County,1,1767.25,1777.50,1750.0,1728.5,...,2392.5,2345.0,2347.5,2347.5,2343.5,2433.0,2334.5,1985.0,1950.0,1725.0


In [18]:
#Assumed Last Column is Target, it should return Xtrain,Ytrain,Xtest,Ytest
def ChangeDataSetForTimeSeries(TheDF):
    posOfFirstDate = np.where(TheDF.columns=='Bedrooms')[0][0] + 1
    FirstDate = TheDF.columns[posOfFirstDate]
    T = 10
    Startindex = np.where(TheDF.columns==FirstDate)[0][0]
    #print(TheDF.columns[Startindex])
    TrainEndindex = TheDF.columns.size-3
    #ValidationEndindex = TheDF.columns.size-3
    TestEndindex = TheDF.columns.size-2
    #print(TheDF.columns[TrainEndindex])
    size = TrainEndindex - Startindex + 1
    colListTrain = TheDF.columns[0:Startindex].tolist()
    for i in range(T):
        colListTrain.append('T'+str(i))
    TrainDFX = pd.DataFrame(columns=colListTrain[:-1])
    TrainDFY = pd.DataFrame(columns=['T'+str(T)])
    #print(colListTrain,len(colListTrain))
    #Make Train-------------------------------
    X_Arr = []
    Y_Arr = []
    for i in range(len(TheDF.index)): #Row Iteration
        if(i%25==0):
            print('\r', 'Iteration', i+1, ' / Rows:', len(TheDF.index), end='')
        initdataToInsert = TheDF.iloc[i,0:Startindex].to_numpy().tolist()
        #print('-')
        #print(initdataToInsert)
        #print('-')
        for t in range(Startindex,TrainEndindex + 1 - T):
            x = TheDF.iloc[i,t:t+T].to_numpy().tolist()
            #print(x)
            #x = np.concatenate((initdataToInsert,x))
            X_Arr.extend(initdataToInsert)
            X_Arr.extend(x)
            y = TheDF.iloc[i,t+T]
            Y_Arr.append(y)
            #return X_Arr,Y_Arr
        # X_Arr = np.array(X_Arr).reshape(-1,len(colListTrain))
        # Y_Arr = np.array(Y_Arr).reshape(-1,1)
        
        # X_Arr = pd.DataFrame(X_Arr,columns=colListTrain).to_dict()
        # Y_Arr = pd.DataFrame(Y_Arr,columns=['T10'])
        # #print(X_Arr)
        # TrainDFX = TrainDFX.append(X_Arr,ignore_index=True)
        # TrainDFY = TrainDFY.append(Y_Arr,ignore_index=True)
        #break
        #return TrainDFX,TrainDFY
        #for ind in ():
            #print(X)
            #FinalDFX.loc[ind] = X[:,ind]
            #FinalDFY.loc[ind] = Y.reshape(-1,1)
            #break
        #break   
    print('\r', 'Iteration', i+1, ' / Rows:', len(TheDF.index), end='') 
    X_Arr = np.array(X_Arr).reshape(-1,len(colListTrain))
    Y_Arr = np.array(Y_Arr).reshape(-1,1)
    
    X_Arr = pd.DataFrame(X_Arr,columns=colListTrain)
    Y_Arr = pd.DataFrame(Y_Arr,columns=['T10'])
    #print(X_Arr)
    TrainDFX = TrainDFX.append(X_Arr,ignore_index=True)
    TrainDFY = TrainDFY.append(Y_Arr,ignore_index=True)
    #Training Sets Done!!!!!
    testColNames = TheDF.columns[0:Startindex].to_numpy().tolist()
    #valColData = TheDF.columns[0:Startindex].to_numpy().tolist()
    testColData = TheDF.columns[0:Startindex].to_numpy().tolist()
    for i in range(T):
        testColNames.append('T'+str(i))
        #valColData.extend([TheDF.columns[ValidationEndindex-T+i+1]])
        testColData.extend([TheDF.columns[TestEndindex-T+i+1]])
    #Get Data of Validation and Testing.
    print('----')
    #print(valColData,testColData)
    #ValDFX = TheDF[valColData].copy()
    #ValDFX.columns = valAndTestColNames
    #ValDFY = TheDF.iloc[:,ValidationEndindex+1].copy()
    #ValDFY.columns = 'T'+str(T)
    # ValidationEndindex
    TestDFX = TheDF[testColData].copy()
    TestDFX.columns = testColNames
    TestDFY = TheDF.iloc[:,TestEndindex+1].copy()
    TestDFY.columns = 'T'+str(T)
    #return TrainDFX,TrainDFY,ValDFX,ValDFY,TestDFX,TestDFY
    return TrainDFX,TrainDFY,TestDFX,TestDFY
    pass

In [19]:
TrainDFX,TrainDFY,TestDFX,TestDFY = ChangeDataSetForTimeSeries(DF)

 Iteration 3389  / Rows: 3389----


In [20]:
TrainDFX

,BedroomsSq,RegionName,State,Metro,CountyName,Bedrooms,T0,T1,T2,T3,T4,T5,T6,T7,T8,T9
0,2,Springfield,OH,Springfield,Clark County,2,827.5,860.0,812.5,850.0,867.5,875.0,875.0,900.0,877.5,900.0
1,2,Springfield,OH,Springfield,Clark County,2,860.0,812.5,850.0,867.5,875.0,875.0,900.0,877.5,900.0,890.0
2,2,Springfield,OH,Springfield,Clark County,2,812.5,850.0,867.5,875.0,875.0,900.0,877.5,900.0,890.0,600.0
3,2,Springfield,OH,Springfield,Clark County,2,850.0,867.5,875.0,875.0,900.0,877.5,900.0,890.0,600.0,600.0
4,2,Springfield,OH,Springfield,Clark County,2,867.5,875.0,875.0,900.0,877.5,900.0,890.0,600.0,600.0,612.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57608,1,Wyoming,MI,Grand Rapids-Wyoming,Kent County,1,995.0,962.5,831.0,849.0,832.0,847.5,821.0,828.5,838.0,844.0
57609,1,Wyoming,MI,Grand Rapids-Wyoming,Kent County,1,962.5,831.0,849.0,832.0,847.5,821.0,828.5,838.0,844.0,858.0
57610,1,Wyoming,MI,Grand Rapids-Wyoming,Kent County,1,831.0,849.0,832.0,847.5,821.0,828.5,838.0,844.0,858.0,876.5
57611,1,Wyoming,MI,Grand Rapids-Wyoming,Kent County,1,849.0,832.0,847.5,821.0,828.5,838.0,844.0,858.0,876.5,891.0


In [21]:
del DF

In [22]:
#lb = LabelEncoder()

In [23]:
#Found out Bedroom was more correlated than these and then on 2nd place is State
# TrainDF['RegionName'] = lb.fit_transform(TrainDF['RegionName'])
# TrainDF['State'] = lb.fit_transform(TrainDF['State'])
# TrainDF['Metro'] = lb.fit_transform(TrainDF['Metro'])
# TrainDF['CountyName'] = lb.fit_transform(TrainDF['CountyName'])
#TrainDF['State'] = lb.fit_transform(TrainDF['State'])

In [24]:
#TrainDF.corr()

In [25]:
#Pass by Value Reference Magic!
def AddStateSparseMatrix(TrainingDF):
   enc = OneHotEncoder(handle_unknown='ignore',sparse = False)
   SM = enc.fit_transform(TrainingDF['State'].to_numpy().reshape(-1,1))
   for ind,cat in zip(range(0,len(enc.categories_[0])),enc.categories_[0]):
      TrainingDF.insert(0,'is'+str(cat),SM[:,ind])
   TrainingDF.drop(columns=['State'],inplace=True)

### Run Models

In [26]:
def RunAllModels(TrainDF,Output,title):
    global runNo
    global RecordAnalysisDF
    global isBedStateOptimized
    def PrintMetrics(scores,algo):
        global RecordAnalysisDF
        print('-'*5)
        meanR2 = np.absolute(scores['test_r2']).mean()
        medianR2 = np.median(np.absolute(scores['test_r2']))
        meanRMSE = np.absolute(scores['test_neg_root_mean_squared_error']).mean()
        medianRMSE = np.median(np.absolute(scores['test_neg_root_mean_squared_error']))
        meanMAPE = np.absolute(scores['test_neg_mean_absolute_percentage_error']).mean()
        medianMAPE = np.median(np.absolute(scores['test_neg_mean_absolute_percentage_error']))
        print('Average R2 score ', meanR2)
        print('Median R2 score ', medianR2)
        print('Average Root Mean Square Error ', meanRMSE)
        print('Median Root Mean Square Error ', medianRMSE)
        print('-'*5)
        if(isBedStateOptimized==False):
            algo = algo + ' Nulls removed with complete column'
        RecordAnalysisDF = RecordAnalysisDF.append({'RunNumber':runNo,'Title':title,'Algorithm':algo,'R2Mean':meanR2,'R2Median':medianR2,'RMSEMean':meanRMSE,'RMSEMedian':medianRMSE},ignore_index=True)
        #print('Average Mean Absolute Percentage Error ', meanMAPE)
        #print('Median Mean Absolute Percentage Error ', medianMAPE)
    #-----------------------Linear Regression---------------------------------------------
    print(title)
    print('-'*10)
    print('Linear Regression')
    model = make_pipeline(StandardScaler(),LinearRegression())
    # Y_pred = model.predict(X_test)
    # Y_pred = scY.inverse_transform(Y_pred)
    # Y_test = scY.inverse_transform(Y_test)
    scores = cross_validate(model, TrainDF, Output, cv=15,scoring=('r2', 'neg_root_mean_squared_error','neg_mean_absolute_percentage_error'))
    PrintMetrics(scores,'Linear Regression')#,meanMAPE,medianMAPE)
    #----------------------------Polynomial Lasso---------------------------------------
    for _,degree in enumerate([1]):
        polymodel = make_pipeline(StandardScaler(),PolynomialFeatures(degree),Lasso(alpha=1e-2,max_iter=1e+6,tol=1e-2))
        scores = cross_validate(polymodel, TrainDF, Output, cv=15,scoring=('r2', 'neg_root_mean_squared_error','neg_mean_absolute_percentage_error'))
        print('Polynomial Degree with Lasso: '+str(degree))
        PrintMetrics(scores,'Lasso Degree ' + str(degree))

        polymodel = make_pipeline(StandardScaler(),PolynomialFeatures(degree),Ridge(alpha=1e-3,max_iter=1e+6))
        scores = cross_validate(polymodel, TrainDF, Output, cv=15,scoring=('r2', 'neg_root_mean_squared_error','neg_mean_absolute_percentage_error'))
        print('Polynomial Degree with Ridge: '+str(degree))
        PrintMetrics(scores,'Ridge Degree ' + str(degree))
    #----------------------------DecisionTreeRegressor---------------------------------------
    model = make_pipeline(StandardScaler(),DecisionTreeRegressor(max_depth=100))
    scores = cross_validate(model, TrainDF, Output, cv=15,scoring=('r2', 'neg_root_mean_squared_error','neg_mean_absolute_percentage_error'))
    print('DecisionTreeRegressor')
    PrintMetrics(scores,'DecisionTreeRegressor')
    runNo = runNo + 1
    print('-'*5)
    print('-'*15)

### Method 1: Drop all the Categorical columns

In [27]:
FinalTrainDF = TrainDFX.drop(columns=['RegionName', 'Metro', 'CountyName'])
# RemoveNulls(FinalTrainDF,True)
FinalTrainDF.drop(columns=['State'],inplace=True)
RunAllModels(FinalTrainDF,TrainDFY,'Dropped Categorical Data')

Dropped Categorical Data
----------
Linear Regression
-----
Average R2 score  0.9849928561541622
Median R2 score  0.9901707051212374
Average Root Mean Square Error  158.0067710715298
Median Root Mean Square Error  129.8187608731687
-----
Polynomial Degree with Lasso: 1
-----
Average R2 score  0.9790593644677237
Median R2 score  0.9854730928991886
Average Root Mean Square Error  182.03409840190267
Median Root Mean Square Error  144.22921422235459
-----
Polynomial Degree with Ridge: 1
-----
Average R2 score  0.9849929479395059
Median R2 score  0.9901709154924679
Average Root Mean Square Error  158.00644804614336
Median Root Mean Square Error  129.81746566475977
-----
DecisionTreeRegressor
-----
Average R2 score  0.9648915348262006
Median R2 score  0.9774879678754967
Average Root Mean Square Error  264.34587047411645
Median Root Mean Square Error  182.82950249171918
-----
-----
---------------


### Method2: Next, Let's change States to a sparse matrix and then to 51 columns, 1 for each State. Lets drop the other categorical columns

In [28]:
FinalTrainDF = TrainDFX.drop(columns=['RegionName', 'Metro', 'CountyName'])
#RemoveNulls(FinalTrainDF,True)
AddStateSparseMatrix(FinalTrainDF)
#FinalTrainDF.drop(columns=['State'],inplace=True)
RunAllModels(FinalTrainDF,TrainDFY,'One Hot Encoded State,Removed Others')
#FinalTrainDF

One Hot Encoded State,Removed Others
----------
Linear Regression
-----
Average R2 score  0.985024411920578
Median R2 score  0.9902466648840058
Average Root Mean Square Error  157.93357695682107
Median Root Mean Square Error  129.60280984773328
-----
Polynomial Degree with Lasso: 1
-----
Average R2 score  0.9790702934269788
Median R2 score  0.9855457624388688
Average Root Mean Square Error  182.05421950135653
Median Root Mean Square Error  145.56754866994018
-----
Polynomial Degree with Ridge: 1
-----
Average R2 score  0.98502396890015
Median R2 score  0.9902467230558996
Average Root Mean Square Error  157.9353752887838
Median Root Mean Square Error  129.6037279938246
-----
DecisionTreeRegressor
-----
Average R2 score  0.9629874241341022
Median R2 score  0.9736511455051692
Average Root Mean Square Error  282.72789781772536
Median Root Mean Square Error  186.57353226144994
-----
-----
---------------


In [29]:
RecordAnalysisDF

,RunNumber,Title,Algorithm,R2Mean,R2Median,RMSEMean,RMSEMedian
0,1,Dropped Categorical Data,Linear Regression,0.984993,0.990171,158.006771,129.818761
1,1,Dropped Categorical Data,Lasso Degree 1,0.979059,0.985473,182.034098,144.229214
2,1,Dropped Categorical Data,Ridge Degree 1,0.984993,0.990171,158.006448,129.817466
3,1,Dropped Categorical Data,DecisionTreeRegressor,0.964892,0.977488,264.345870,182.829502
4,2,"One Hot Encoded State,Removed Others",Linear Regression,0.985024,0.990247,157.933577,129.602810
5,2,"One Hot Encoded State,Removed Others",Lasso Degree 1,0.979070,0.985546,182.054220,145.567549
6,2,"One Hot Encoded State,Removed Others",Ridge Degree 1,0.985024,0.990247,157.935375,129.603728
7,2,"One Hot Encoded State,Removed Others",DecisionTreeRegressor,0.962987,0.973651,282.727898,186.573532


In [30]:
RecordAnalysisDF.index +=1

In [31]:
RecordAnalysisDF.to_csv('RecordAnalysis.csv',index_label='S.No.')

In [32]:
#After Analyzing this CSV File (Copy of the table is in the report), we found that "One Hot Encoded State,Removed Others - NAN removed with column with Linear Regression" had the least R2Mean and RMSE Median. So, it had performed best in the 2/4 metrics we considered. Hence, we decided to go ahead with it.

### Testing

In [42]:
#One Hot Encoded State,Removed Others	Linear Regression
FinalTrainDF = TrainDFX.drop(columns=['RegionName', 'Metro', 'CountyName'])
#RemoveNulls(FinalTrainDF,True)
AddStateSparseMatrix(FinalTrainDF)
FinalTestDF = TestDFX.drop(columns=['RegionName', 'Metro', 'CountyName'])
#FinalTrainDF[ValDF.name] = ValDF
#RemoveNulls(FinalTrainDF,False)
AddStateSparseMatrix(FinalTestDF)
#FinalTrainDF.drop(columns=['State'],inplace=True)

In [43]:
model = LinearRegression()
hist = model.fit(FinalTrainDF,TrainDFY)
YPred = model.predict(FinalTestDF)

In [44]:
R2Score = r2score(TestDFY,YPred)
RMSE = mse(TestDFY,YPred,squared=False)
print('R2 score ', R2Score)
print('Root Mean Square Error ', RMSE)
print('Average of Output ',TestDFY.mean())

R2 score  0.9944659122041594
Root Mean Square Error  136.35425137459862
Average of Output  1779.1479787547948


In [45]:
136.356818823521/1779.1479787547948

0.07664163996013168

In [46]:
lb.classes_

array(['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
       'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
       'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
       'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
       'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY'], dtype=object)

In [47]:
for bed in range(1,5):
    filterArr = (FinalTestDF['Bedrooms'] == bed)
    #print(filterArr)
    print('For '+ str(bed) +' bedrooms')
    print('-'*5)
    R2Score = r2score(TestDFY[filterArr],YPred[filterArr])
    RMSE = mse(TestDFY[filterArr],YPred[filterArr],squared=False)
    print('R2 score ', R2Score)
    print('Root Mean Square Error ', RMSE)
    print('Average of '+ str(bed) +' Bedroom Cost ',TestDFY[filterArr].mean())
    print('Number of Houses ',len(TestDFY[filterArr].index))
    print('-'*10)

For 1 bedrooms
-----
R2 score  0.9913823818975729
Root Mean Square Error  49.12731717848224
Average of 1 Bedroom Cost  1384.4834963325184
Number of Houses  818
----------
For 2 bedrooms
-----
R2 score  0.9922284242596486
Root Mean Square Error  59.51218426454646
Average of 2 Bedroom Cost  1566.38814229249
Number of Houses  1265
----------
For 3 bedrooms
-----
R2 score  0.996303893085298
Root Mean Square Error  96.80884066962692
Average of 3 Bedroom Cost  1969.8116161616163
Number of Houses  990
----------
For 4 bedrooms
-----
R2 score  0.9935543538200522
Root Mean Square Error  386.7963223920657
Average of 4 Bedroom Cost  3055.159810126582
Number of Houses  316
----------
